In [18]:
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from features import speech_feat_extract
import numpy as np
import pickle

sess = ['Session'+str(i+1) for i in range(5)]

folderpath = '../../../../../Other/IEMOCAP_full_release/' # specify the path to dataset
data = speech_feat_extract(sess, folderpath)

4936


In [13]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Activation
import numpy as np
import pickle

# replace the features with the required features file
#with open('utterance_feat/AE_feat_lpcc_80_256.pkl') as f:
#    X = pickle.load(f)
#tr_x = X['feat']['train']
#te_x = X['feat']['test']
#tr_y = X['cat']['train']
#te_y = X['cat']['test']

# use it only for voice quality features
with open('feat/80_20/voiceQuality.pkl') as f:
    X = pickle.load(f)
tr_x = np.concatenate((tr_x, X['feat']['train']), axis=1)
te_x = np.concatenate((te_x, X['feat']['test']), axis=1)

# make fusion_nn True if you want to concatenate any features
fusion_nn = False
if fusion_nn:
    from sklearn.preprocessing import label_binarize
    tr_yc = label_binarize(tr_y, list(set([i for i in tr_y])))
    te_yc = label_binarize(te_y, list(set([i for i in te_y])))
    nn_comb = nn_feat_fusion(tr_x.shape[1])
    print nn_comb.train(tr_x, tr_yc)
    tr_x = nn_comb.feature(tr_x)
    te_x = nn_comb.feature(te_x)
    with open('combined_256_mfcc_VQ_lpcc.pkl', 'wb') as f:
        pickle.dump({'feat':{'train':tr_x, 'test':te_x}, 'cat':{'train':tr_y, 'test':te_y}}, f)

clf = svm.SVC(kernel='poly', random_state=1)
clf.fit(tr_x, tr_y)
pred = clf.predict(te_x)
pred_train = clf.predict(tr_x)
print accuracy_score(te_y, pred)
print accuracy_score(tr_y, pred_train)

from sklearn.metrics import confusion_matrix
cnf = confusion_matrix(te_y, pred)
print cnf

0.3866396761133603
0.37031408308004055
[[ 22   0 139   0]
 [ 31   0 227   0]
 [  9   0 354   2]
 [ 13   0 185   6]]


In [ ]:
# make the flag True in case labels are categorical, used in older version
check = False
if check:
    tr_y = []
    te_y = []
    for i in list(X['cat']['train']):
        if tuple(i) == (1,0,0,0):
            tr_y.append(1)
        if tuple(i) == (0,1,0,0):
            tr_y.append(2)
        if tuple(i) == (0,0,1,0):
            tr_y.append(3)
        if tuple(i) == (0,0,0,1):
            tr_y.append(4)
    for i in list(X['cat']['test']):
        if tuple(i) == (1,0,0,0):
            te_y.append(1)
        if tuple(i) == (0,1,0,0):
            te_y.append(2)
        if tuple(i) == (0,0,1,0):
            te_y.append(3)
        if tuple(i) == (0,0,0,1):
            te_y.append(4)
    tr_yc = np.array(tr_y)
    te_yc = np.array(te_y)